1. 使用python获取第一页html内容并解码为文本串，以utf8编码格式写入page1.txt

https://www.nature.com/search?q=llm&order=relevance&date_range=2023-2024

In [1]:
import requests
from bs4 import BeautifulSoup

url ="https://www.nature.com/search?q=llm&order=relevance&date_range=2023-2024"
response = requests.get(url)

if response.status_code == 200:
    html_content=response.text
    file_path="page1.txt"
    with open(file_path,"w",encoding="utf-8") as file:
        file.write(html_content)
    

2. 打开page1.txt，提取论文相关信息（标题，地址，简介，作者列表，文章类型，期刊名称，卷宗），按照期刊名进行分类

In [2]:
file_path="page1.txt"
with open(file_path,"r",encoding="utf-8") as file:
    content=file.read()
soup=BeautifulSoup(content,'html.parser')
journal_dict={}#初始化字典
papers=soup.find_all("article")
for paper in papers:
    # 提取文章标题
    title = paper.find("h3").text.strip()
    
    # 提取文章地址
    url = paper.find("a")["href"]
    
    # 提取文章简介
    description = paper.find("p")
    if description:
        description = description.text.strip()
    else:
        description = "no description"

    # 提取作者列表
    authors = paper.find_all("span", itemprop="name")
    if authors:
        authors = [author.text.strip() for author in authors]
    else:
        authors = []
    # 提取文章类型
    type_element= paper.find("span", class_="c-meta__type")
    if type_element:
        type=type_element.text.strip()
    else:
        type=""

    # 提取期刊名称
    journal_element = paper.find("div", class_="c-meta__item c-meta__item--block-at-lg u-text-bold")
    if journal_element:
        journal=journal_element.text.strip()
    else:
        journal=""

    # 提取卷宗/页面信息
    volume_page_info = paper.find("div", class_="c-meta__item c-meta__item--block-at-lg")
    if volume_page_info:
        volume_page_info = volume_page_info.text.strip()
    else:
        volume_page_info = ""
    if journal in journal_dict:
        journal_dict[journal]["papers"].append({
            "title": title,
            "authors": authors,
            "url": url,
            "description": description,
            "type": type,
            "volume_page_info": volume_page_info
        })
    else:
        journal_dict[journal] = {
            "journal": journal,
            "papers": [{
                "title": title,
                "authors": authors,
                "url": url,
                "description": description,
                "type": type,
                "volume_page_info": volume_page_info
            }]
        }
for journal in journal_dict.values():
    print(f"期刊 {journal['journal']} 包含 {len(journal['papers'])} 篇论文")

期刊 Nature Machine Intelligence 包含 6 篇论文
期刊 Nature Communications 包含 10 篇论文
期刊 Scientific Reports 包含 12 篇论文
期刊 Nature Methods 包含 1 篇论文
期刊 npj Digital Medicine 包含 3 篇论文
期刊 Nature Medicine 包含 2 篇论文
期刊 Nature 包含 4 篇论文
期刊 Nature Human Behaviour 包含 2 篇论文
期刊 npj Precision Oncology 包含 1 篇论文
期刊 Nature Computational Science 包含 2 篇论文
期刊 BDJ Open 包含 1 篇论文
期刊 Nature Reviews Urology 包含 1 篇论文
期刊 Communications Materials 包含 1 篇论文
期刊 Humanities and Social Sciences Communications 包含 1 篇论文
期刊 npj Biodiversity 包含 1 篇论文
期刊 Eye 包含 1 篇论文
期刊 npj Computational Materials 包含 1 篇论文


#### bs4中find()和find_all()可能的报错

`title = paper.find("h3").text.strip()`

如果paper.find("h3")未查找到，则会返回none
而none被text作用会报错

**解决方法**：添加if检查
#### find()用法
不要直接使用AI生成的代码，应该直接查找html格式中具体每个标签是什么

3. 将上一步字典中提取内容中的作者列表中的内容替换为文章主页显示的全部作者。

基于上述结果，输出每个期刊在page1中包含的论文数量

从page1.txt中提取每篇文章的链接，从链接中获取全部作者，替换journal_dict中每篇文章的作者。

In [ ]:
base_url="https://www.nature.com"
for journal1 in journal_dict:
    for paper in journal_dict[journal1]['papers']:
        paper_url=paper['url']
        full_url=base_url+paper_url
        response = requests.get(full_url)
        if response.status_code == 200:
            html_content=response.text
            soup=BeautifulSoup(html_content,'html.parser')
            authors_meta = soup.find_all('meta', {'name': 'citation_author'})
            if authors_meta:
                authors = [f"{meta['content'].split(', ')[1]} {meta['content'].split(', ')[0]}" 
                      for meta in authors_meta]
                paper["authors"]=authors  

**如何遍历各个论文中的url？**
1. 首先对`journal_dict`遍历，得到各个期刊名字的序号
2. 再对journal_dict`[journal1]['papers']`遍历，其中`journal1`是第一步中的变量，`'papers'`为字典中储存各个期刊中的论文的key
3. 此时`paper`(第二部中的变量)['url']，即可根据paper在journal1及journal1在journal_dict这两次遍历跑遍各个论文

In [4]:
import json

with open("nature_llm.json","w",encoding="utf-8") as f:
    json.dump(journal_dict, f, indent=2, ensure_ascii=False)